In [ ]:
# import our packages
import pandas as pd
import requests
import os  
import time
from nba_api.stats.static import players
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import SeasonAll
from nba_api.stats.endpoints import boxscoretraditionalv2
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Get All Player Data 

In [ ]:
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [ ]:
if os.path.exists("./NBA_Stats/player_career_stats.csv"):
  os.remove("./NBA_Stats/player_career_stats.csv")
else:
  print("The file does not exist") 

In [ ]:
player_dictionary = players.get_players()


In [ ]:

for player in player_dictionary:
    try:    
        if player['is_active'] == True:
            print(player['full_name'])
            career = playercareerstats.PlayerCareerStats(player_id = player['id'])
            career.get_data_frames()[0].to_csv(f'./NBA_Stats/player_career_stats.csv', mode='a', index=False, header=True)
            time.sleep(.600)
    except:
        continue

In [ ]:
df = pd.read_csv('./NBA_Stats/player_career_stats.csv')
grouped = df.groupby("PLAYER_ID")
grouped.head()

In [ ]:
df["Season_Count"] = grouped["PLAYER_ID"].cumcount() + 1

df.head()

In [ ]:
# find the indices of the rows that contain the headers
header_rows = [i for i, row in df.iterrows() if row.astype(str).str.contains('PTS|REB|AST').any()]

# drop the rows that contain the headers
df.drop(header_rows, inplace=True)
df.to_csv('./NBA_Stats/player_career_stats_cleaned.csv', index=False)

In [ ]:
df = pd.read_csv('./NBA_Stats/player_career_stats_cleaned.csv')

# Group the data by player name and calculate the cumulative sum of points for each player
df['career_points'] = df.groupby('PLAYER_ID')['PTS'].cumsum()

# Reset the index to ensure that the career points are in the correct order for each player
df = df.reset_index(drop=True)

# Write the result back to a csv file
df.to_csv('./NBA_Stats/player_career_stats_cleaned.csv', index=False)